# IMPORTAMOS LA LIBRERIAS NECESARIAS

In [1]:
from glob import glob
import pandas as pd

### Cargamos la data

In [2]:
df_metadata = pd.read_parquet('../../data/raw/g-sitios.parquet')

In [3]:
# Funsion para hacer un explode en aquellas columnas que contienen listas de datos.
def explode_columna(df: pd.DataFrame, columna_a_explodar: str) -> pd.DataFrame:
    return df.explode(columna_a_explodar)
# Funsion para filtrar la categoria que vayamos a usar de acuerdo al tipo de estudio de eleccion.
def filtrar_por_categoria(df: pd.DataFrame, columna_a_filtrar: str, keyword: str) -> pd.DataFrame:
    df = df.dropna(subset=[columna_a_filtrar])
    return df[df[columna_a_filtrar].str.contains(keyword, case=False)]

In [4]:
df_metadata.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [12]:
df_nuevo = explode_columna(df_metadata,'category')

In [14]:
# Definir el patrón regex para extraer la dirección, ciudad, estado y código postal
patron = r'([^,]+),\s*([^,]+),\s*([^,]+),\s*([^,]+)$'

# Aplicar el regex a cada entrada de la columna "address" y almacenar los resultados en columnas separadas
resultados = df_nuevo['address'].str.extract(patron)

# Asignar los resultados a nuevas columnas en el DataFrame
df_nuevo['Local'] = resultados[0]
df_nuevo['Direccion'] = resultados[1]
df_nuevo['Ciudad'] = resultados[2]
df_nuevo['Codigo_Postal'] = resultados[3]

# Insertar las nuevas columnas justo después de la columna "address"
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 1, 'Local', df_nuevo.pop('Local'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 2, 'Direccion', df_nuevo.pop('Direccion'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 3, 'Ciudad', df_nuevo.pop('Ciudad'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 4, 'Codigo_Postal', df_nuevo.pop('Codigo_Postal'))

In [15]:
df_nuevo.head(2)

,name,address,Local,Direccion,Ciudad,Codigo_Postal,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,Estado
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy,129 N Second St,Cochran,GA 31014,0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,Pharmacy,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...,Cochran
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile,3001 E Pico Blvd,Los Angeles,CA 90023,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,Textile exporter,4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,Los Angeles


In [16]:
df_nuevo.drop(columns=["address"], inplace=True)

In [17]:
df_nuevo.drop(columns=["Local"], inplace=True)

In [18]:
# Dividir la columna "Codigo_Postal" en dos nuevas columnas: "Estado" y "Codigo_Postal"
df_nuevo[['Estado', 'Codigo_Postal']] = df_nuevo['Codigo_Postal'].str.split(' ', n=1, expand=True)

In [19]:
df_nuevo.head(5)

,name,Direccion,Ciudad,Codigo_Postal,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,Estado
0,Porter Pharmacy,129 N Second St,Cochran,31014,0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,Pharmacy,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...,GA
1,City Textile,3001 E Pico Blvd,Los Angeles,90023,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,Textile exporter,4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
2,San Soo Dang,761 S Vermont Ave,Los Angeles,90005,0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,Korean restaurant,4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
3,Nova Fabrics,2200 E 11th St,Los Angeles,90021,0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,Fabric store,3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
4,Nobel Textile Co,719 E 9th St,Los Angeles,90021,0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,Fabric store,4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
